In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression


import warnings
warnings.filterwarnings('ignore')


In [2]:
mercado_df = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Mercado/data_consolidado_final_jun24.csv",
                     encoding='latin-1',
                     delimiter=';')

In [3]:
df_mod=mercado_df[["premio_total_liq_cap","ano_mod_conv","classe_bonus",
                   "idade","is_casco","is_dmoral","is_rcdc","is_rcdm",
                   "RNS_Casco_calc",
                   "Grupo_veiculo_new", "fabrica","idade_ult_RNS","md_categoria",
                   "MD_comb","md_config","MD_franquia","md_menor26","md_sexo",
                   "MD_Subzona","md_tipo_renov","MD_uso_veic","uf"]]

df_mod.rename(columns={'premio_total_liq_cap':'premio_ofertado', 'ano_mod_conv': 'ano_mod'},inplace=True)

In [4]:
x_df_num = df_mod[['ano_mod', 'classe_bonus', 'idade', 'is_casco', 'is_dmoral', 'is_rcdc', 'is_rcdm', 'RNS_Casco_calc']]
x_df_cat = df_mod[['Grupo_veiculo_new', 'fabrica', 'idade_ult_RNS', 'md_categoria', 'MD_comb', 'md_config', 'MD_franquia', 'md_menor26', 'md_sexo', 'MD_Subzona','md_tipo_renov', 'MD_uso_veic','uf']]
y   = df_mod[['premio_ofertado']]
x_df_dummy=pd.get_dummies(x_df_cat)
x_df=pd.concat([x_df_num, x_df_dummy],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.15) 

In [9]:
import lightgbm as lgb
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
}

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval  = lgb.Dataset(x_test, y_test, reference=lgb_train)

model_gbm = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval)

y_pred = model_gbm.predict(x_test, predict_disable_shape_check=True)
print("Premio Estimado Médio: %.2f" % np.mean(y_pred) )
# accuracy check
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

Premio Estimado Médio: 2632.46
MSE: 631777.76
RMSE: 794.84


## modelagem GLM

In [10]:
df_mod_mp_feb23may24_v3 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23may24_v3.csv",
                     encoding='latin-1',
                     delimiter='|')

parte para incluir o preço do mercado na base de modelagem glm

In [11]:
colunas=['premio_ofertado','ano_mod','classe_bonus','MD_idade','is_casco_ofer','is_dmoral','is_rcdc','is_rcdm','RNS_Casco_calc',
 'Grupo_veiculo_new','FABRICA','idade_ult_RNS', 'md_categoria','MD_comb','MD_config','MD_franquia','MD_menor26','MD_sexo','MD_Subzona',  
 'MD_tipo_renov','MD_uso_veic','UF']

In [12]:
df_escorar_gbm=df_mod_mp_feb23may24_v3[colunas]

In [13]:
df_escorar_gbm.rename(columns={'MD_idade':'idade', 
                                   'is_casco_ofer': 'is_casco',
                                   'FABRICA': 'fabrica',
                                   'MD_config':  'md_config',
                                   'MD_menor26': 'md_menor26',
                                   'MD_sexo': 'md_sexo',
                                   'UF': 'uf',
                                   'MD_tipo_renov':  'md_tipo_renov'},inplace=True)
df_escorar_gbm['idade'] = pd.to_numeric(df_escorar_gbm['idade'], errors='coerce')

In [14]:
df_num = df_escorar_gbm[['ano_mod', 'classe_bonus', 'idade', 'is_casco', 'is_dmoral', 'is_rcdc', 'is_rcdm', 'RNS_Casco_calc']]
df_cat = df_escorar_gbm[['Grupo_veiculo_new', 'fabrica', 'idade_ult_RNS', 'md_categoria', 'MD_comb', 'md_config', 'MD_franquia', 'md_menor26', 'md_sexo', 'MD_Subzona','md_tipo_renov', 'MD_uso_veic','uf']]
df_cat_glm=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_num, df_cat_glm],axis=1)
GBM_mercado = model_gbm.predict(df_cat_glm, predict_disable_shape_check=True)

In [16]:
import numpy as np
np.average(GBM_mercado)

4194.9834619682615

In [17]:
df_mod_mp_feb23may24_v3['GBM_mercado']=GBM_mercado

#### prepara base de modelagem glm

In [18]:
df_mod_filtered=df_mod_mp_feb23may24_v3[["premio_ofertado","premio_total",'GBM_mercado', 'premio_creserva','premio_demais_cob',"teste_validacao","ind_renovacao","classe_bonus","MD_sexo","cod_corr",
                                "Cod_End","MD_sin_5a_cas","MD_tempo_casa_2","MD_idade_veic","MD_faixa_IScas",
                                "anomes_dt_ter" ,"MD_idade" ,"per_com", "RNS_Casco_calc","MD_menor26","UF","Renovacao_ant",
                                "fx_ir_12m","is_casco_ofer","classe_bonus","MD_fator_ajuste","MD_capac","md_categoria",
                                "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA','MD_est_civil',
                                'peso','Grupo_veiculo_new','vigencia',
                                'Risco_PP_mod', 'Risco_PT_mod','Risco_RF_mod','Risco_DM_mod','Risco_AS_mod',
                                'Risco_DC_mod' ,'Risco_DMO_mod']]

In [19]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['peso'].notna()]
df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']>0) & (df_mod_filtered['per_com']<50)]
df_mod_filtered = df_mod_filtered.query('premio_total >100')
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
df_mod_filtered = df_mod_filtered.query('is_casco_ofer > 0')
df_mod_filtered.shape

(388318, 45)

In [20]:
df_mod_filtered['DA_emissao'] =0.2349
df_mod_filtered['DA_producao'] =0.2749
df_mod_filtered['risco_dmais_cob'] =df_mod_filtered['premio_demais_cob']*0.2627*1.4
df_mod_filtered['premio_anterior'] =df_mod_filtered['premio_total']
df_mod_filtered['Act_cost']  = df_mod_filtered['Risco_PP_mod'] + df_mod_filtered['Risco_PT_mod'] + df_mod_filtered['Risco_RF_mod']+ df_mod_filtered['Risco_DM_mod']+ df_mod_filtered['Risco_AS_mod'] + df_mod_filtered['Risco_DC_mod'] +df_mod_filtered['Risco_DMO_mod'] + df_mod_filtered['risco_dmais_cob']
df_mod_filtered['Premio_IC100']  = df_mod_filtered['Act_cost']/(1-df_mod_filtered['per_com']/100-df_mod_filtered['DA_producao'] )
df_mod_filtered["premio"] = df_mod_filtered["premio_ofertado"]

In [23]:
df_mod=df_mod_filtered.copy()

In [24]:
def categorizar(df):
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01 - 00-<=020' if x == '01 - 00-<=020' else
                                                              '02 - 20-<=040' if x == '02 - 20-<=040' else
                                                              '03 - 40-<=060' if x == '03 - 40-<=060' else
                                                              '04 - 60-<=080' if x == '04 - 60-<=080' else
                                                              '05 - 80-<=100' if x == '05 - 80-<=100' else
                                                              '02 - 20-<=040' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor Mais' if x == 'Corretor Mais'  else 'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: '<10' if x < 10 else
                                                          '>=10' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x: 'Sem Sinistro' if x == 'Sem Sinistro' else
                                                                      '01 ano' if x == '01 ano' else
                                                                      '02 ano' if x == '02 ano' else '02 ano +')




    df['UF_bin'] = df['UF'].apply(lambda x: 'AL,CE,RS'  if x == 'AL' else
                                                'AL,CE,RS'  if x == 'CE' else
                                                'AL,CE,RS'  if x == 'RS' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'AP' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'DF' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'ES' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'PA' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'PB' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'RN' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'SC' else 
                                                'GO,MA,MG'             if x == 'GO' else 
                                                'GO,MA,MG'             if x == 'MA' else 
                                                'GO,MA,MG'             if x == 'MG' else 
                                                'MT,PR'                if x == 'MT' else
                                                'MT,PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x: '1-SeguroNovo'                      if x == '1-Seguro Novo'    else
                                                                          '2 - Renov MAPFRE sem sinistro'     if x == '2-RenovPropria'   else
                                                                          '3 - Renov MAPFRE com sinistro'     if x == '2-RenovPropria'   else
                                                                          '4 - Renov Congenere sem sinistro'  if x == '3-RenovCongenere' else
                                                                          '5 - Renov Congenere com sinistro'  if x == '3-RenovCongenere' else
                                                                          '6 - Renov BB sem sinistro'         if x == '3-RenovCongenere' else
                                                                          '7 - Renov BB com sinistro'         if x == '3-RenovCongenere' else  '2-RenovPropria')
      
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                          '13509'  if x == 50007 else
                                                          '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: '<101'    if x < 101 else
                                                                       '>=101'   if x >= 101  else '<101')   

    df['MD_qtd_assis_cpf_bin'] = df['MD_qtd_assis_cpf'].apply(lambda x: '00'  if x == 0 else
                                                                            '01'  if x == 1 else
                                                                            '>=2' if x >= 2 else 'ERROR')
    return df

In [25]:
df_mod = categorizar(df_mod)

In [26]:
df_train = df_mod.query('teste_validacao == "Trein"')
df_train.shape

(310913, 64)

In [ ]:
#df_train2 = df_train[df_train[(df_train['UF'] == 'SP')] & (~df_train['Grupo_veiculo_new'].isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
#df_train2.shape

In [ ]:
#df_train = df_train[(df_train['UF'] == 'SP') & (~df_train['Grupo_veiculo_new'].isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
#df_train.shape

In [27]:
display(df_train[["Act_cost", "Premio_IC100", "per_com","ind_renovacao",'GBM_mercado']].describe().T)

,count,mean,std,min,25%,50%,75%,max
Act_cost,310913.0,1723.935999,1533.558565,155.790548,880.102121,1287.055629,2035.829042,74027.191185
Premio_IC100,310913.0,3024.359106,2669.004613,312.305413,1555.405520,2256.816147,3576.591487,124394.540725
per_com,310913.0,15.434452,5.672286,1.000000,10.000000,15.000000,20.000000,49.000000
ind_renovacao,310913.0,0.464191,0.498717,0.000000,0.000000,0.000000,1.000000,1.000000
GBM_mercado,310913.0,4200.978157,1027.088290,2348.154937,3447.642184,4024.520592,4715.245579,11122.115252


In [ ]:
#df_train.columns.to_list()

In [28]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
formula = 'ind_renovacao ~ 1 + np.log(premio) + np.divide(premio, is_casco_ofer) + np.log(np.divide(premio, GBM_mercado)) + np.log(np.divide(premio, Premio_IC100)) + (np.divide(premio, premio_anterior)+1) + C(canal_bin,Treatment(reference="Demais")) + C(Cod_End_bin,Treatment(reference="Demais")) + C(fator_ajuste_bin,Treatment(reference="<101")) + per_com + C(per_com_bin,Treatment(reference=">=10")) + np.log(MD_tempo_casa_2+1) + C(idade_ult_RNS_bin,Treatment(reference="Sem Sinistro")) + C(MD_comb_bin,Treatment(reference="Demais")) + C(UF_bin,Treatment(reference="Demais")) + C(fx_ir_12m_bin,Treatment(reference="02 - 20-<=040")) + C(MD_tipo_renov,Treatment(reference="2 - Renov MAPFRE sem sinistro"))' 
model = smf.logit(formula , data=df_train).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.564716
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          ind_renovacao   No. Observations:               310913
Model:                          Logit   Df Residuals:                   310888
Method:                           MLE   Df Model:                           24
Date:                Sun, 28 Jul 2024   Pseudo R-squ.:                  0.1823
Time:                        18:09:13   Log-Likelihood:            -1.7558e+05
converged:                       True   LL-Null:                   -2.1471e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===========================================================================================================================================================================
                                                                                                              coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                   1.7343      0.211      8.201      0.000       1.320       2.149
C(canal_bin, Treatment(reference="Demais"))[T.Corretor Mais]                                                0.0329      0.010      3.195      0.001       0.013       0.053
C(Cod_End_bin, Treatment(reference="Demais"))[T.13509]                                                     -0.2774      0.014    -20.318      0.000      -0.304      -0.251
C(fator_ajuste_bin, Treatment(reference="<101"))[T.>=101]                                                   0.0809      0.029      2.800      0.005       0.024       0.138
C(per_com_bin, Treatment(reference=">=10"))[T.<10]                                                          0.1816      0.024      7.543      0.000       0.134       0.229
C(idade_ult_RNS_bin, Treatment(reference="Sem Sinistro"))[T.01 ano]                                        -0.5195      0.024    -21.295      0.000      -0.567      -0.472
C(idade_ult_RNS_bin, Treatment(reference="Sem Sinistro"))[T.02 ano +]                                      -0.4643      0.012    -37.352      0.000      -0.489      -0.440
C(MD_comb_bin, Treatment(reference="Demais"))[T.Gasolina]                                                   0.3844      0.019     20.077      0.000       0.347       0.422
C(UF_bin, Treatment(reference="Demais"))[T.AL,CE,RS]                                                       -0.2157      0.013    -15.987      0.000      -0.242      -0.189
C(UF_bin, Treatment(reference="Demais"))[T.AP,DF,ES,PA,PB,RN,SC]                                           -0.2309      0.014    -16.681      0.000      -0.258      -0.204
C(UF_bin, Treatment(reference="Demais"))[T.GO,MA,MG]                                                       -0.0669      0.015     -4.532      0.000      -0.096      -0.038
C(UF_bin, Treatment(reference="Demais"))[T.MT,PR]                                                          -0.1036      0.014     -7.350      0.000      -0.131      -0.076
C(UF_bin, Treatment(reference="Demais"))[T.RJ]                                                              0.3164      0.021     15.422      0.000       0.276       0.357
C(fx_ir_12m_bin, Treatment(reference="02 - 20-<=040"))[T.01 - 00-<=020]                                    -0.9163      0.017    -52.630      0.000      -0.950      -0.882
C(fx_ir_12m_bin, Treatment(reference="02 - 20-<=040"))[T.03 - 40-<=060]                                     0.4855      0.010     48.792      0.000       0.466       0.505
C(fx_ir_12m_bin, Treatment(reference="02 - 20-<=040"))[T.04 - 60-<=080]                                     1.1635      0.013     86.389      0.0

## Base Para escorar e otimizar

In [29]:
df_lote_092024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Table Base Case Total  - modelo novo.csv",
                     encoding='latin-1',
                     delimiter='|')

In [30]:
df_lote_092024['GBM_mercado'].describe()

count    27304.000000
mean      2662.428740
std       1406.693218
min       1015.431214
25%       1797.549029
50%       2284.675931
75%       3005.559169
max      13353.547185
Name: GBM_mercado, dtype: float64

In [31]:
df_lote_escorar=df_lote_092024.copy()

df_lote_escorar=df_lote_092024[['Premio_oferecido_com_batente','Pr_anterior_TI','Premio_IC100','GBM_mercado',
                                          'Classe_bonus',"MD_sexo","cod_corr","cod_end","MD_tempo_casa_2",
                                          "per_com", "MD_menor26","UF","Renovacao_ant","fx_ir_12m","is_casco_ofer",
                                          "MD_fator_ajuste","md_capac","MD_categoria",
                                          "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA',
                                          'MD_est_civil','Grupo_veiculo_new']]

In [ ]:
#print(df_lote_escorar['UF'].value_counts(ascending=False))

premio|Premio_oferecido_com_batente
is_casco_ofer|is_casco_ofer
GBM_mercado|GBM_mercado
Premio_IC100|Premio_IC100
premio_anterior|Pr_anterior_TI
canal_bin| canal
Cod_End_bin|  cod_end
fator_ajuste_bin|  MD_fator_ajuste
per_com|per_com
MD_tempo_casa_2|MD_tempo_casa_2
idade_ult_RNS_bin|idade_ult_RNS
MD_comb_bin|MD_comb
UF_bin      |     UF
fx_ir_12m_bin|    fx_ir_12m
MD_tipo_renov,|   MD_tipo_renov

In [32]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [33]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [35]:
df_lote_escorar=df_lote_escorar[df_lote_escorar['UF']!= "99"]

In [36]:
df_lote_escorar.shape

(27298, 123)

In [37]:
df_lote = categorizar(df_lote_escorar)

In [38]:
df_lote["prob_est"]=model.predict(df_lote)

In [289]:
df_lote["prob_est"].describe()

count    27298.000000
mean         0.464357
std          0.222866
min          0.017009
25%          0.284619
50%          0.456997
75%          0.624807
max          0.991476
Name: prob_est, dtype: float64

In [41]:
df_lote[["prob_est",'Renewal Demand']].describe()

,prob_est,Renewal Demand
count,27298.000000,27298.000000
mean,0.464357,0.505496
std,0.222866,0.239074
min,0.017009,0.011015
25%,0.284619,0.310497
50%,0.456997,0.514788
75%,0.624807,0.694137
max,0.991476,0.995868


## Otimização Teste 

In [42]:
print(df_lote['Amostra'].value_counts(ascending=False))

Otimização Mkp     15569
Otimização Mkp2     3271
Controle            2830
Otimização Atu      2829
Teste_+5%           1402
Teste_-5%           1397
Name: Amostra, dtype: int64


In [290]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra == "Otimização Atu"')
df_lote_otim_atu.shape

(2829, 136)

In [394]:
df_lote_otim = df_lote_otim_atu

In [429]:
df_lote_otim = df_lote_otim_atu.sample(n = 100) 

In [430]:
#df_lote_otim = df_otim1
df_lote_otim.shape

(100, 140)

In [431]:
df_lote_otim["Margem Inicial"] = df_lote_otim["prob_est"]*(df_lote_otim["Premio_oferecido_com_batente"]-df_lote_otim["Act_cost"])

In [432]:
print('Premio Inicial: %2f' % np.mean(df_lote_otim["premio"]))
print('Probabilidade Inicial: %2f' % np.mean(df_lote_otim["prob_est"]))
print('Margem Inicial: %2f' % np.mean(df_lote_otim["Margem Inicial"]))

Premio Inicial: 2899.420738
Probabilidade Inicial: 0.433084
Margem Inicial: 447.052089


In [433]:
#df_lote_otim2['premio_inicial'] =( (df_lote_otim2["premio"]*0.9) + (df_lote_otim2["premio"]*1.1))/2
#df_lote_otim2

In [449]:
%%time

def margem_esperada(premio,demanda_alvo):
    df_lote_otim["premio"]=premio
    demanda= model.predict(df_lote_otim)
    df_lote_otim["demanda_alvo"]=demanda_alvo
    custo= df_lote_otim["Act_cost"]
    margem_esperada = demanda * (premio-custo)
    margem_total = -np.sum(margem_esperada) 
    return margem_total

# restricão demanda alvo
def constraint(premio,demanda_alvo):
    df_lote_otim["premio"]=premio
    demanda= model.predict(df_lote_otim)
    demanda_alvo = df_lote_otim["demanda_alvo"]
    return  np.sum(demanda) - np.sum(demanda_alvo)


def constraint2(premio,demanda_alvo):
    margem_inicial = df_lote_otim["prob_est"]*(df_lote_otim["Premio_oferecido_com_batente"]-df_lote_otim["Act_cost"])
    df_lote_otim["premio"]=premio
    df_lote_otim["demanda"] = model.predict(df_lote_otim)
    demanda_alvo = df_lote_otim["demanda_alvo"]
    margem_otim = df_lote_otim["demanda"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])
    return  np.sum(margem_otim) - np.sum(margem_inicial)


def constraint3(premio,demanda_alvo):
    lb = df_lote_otim["Premio_oferecido_com_batente"]*0.90
    return lb

def constraint4(premio,demanda_alvo):
    ub = df_lote_otim["Premio_oferecido_com_batente"]*1.1
    return ub
 
restr_demanda = {'type': 'eq', 'fun': constraint}   
restr_margem  = {'type': 'ineq', 'fun': constraint2} 
restr_lb = {'type': 'ineq', 'fun': constraint3}   
restr_ub  = {'type': 'ineq', 'fun': constraint4} 
 
#restricoes = ([restr_demanda,restr_margem,restr_lb,restr_ub])     

restricoes = ([restr_demanda,restr_margem])     

#df_lote_otim['lb'] = df_lote_otim["Premio_oferecido_com_batente"]*0.90
#df_lote_otim['ub']  = df_lote_otim["Premio_oferecido_com_batente"]*1.1
#premio_inicial = (df_lote_otim['lb'].astype('int') +df_lote_otim['ub'].astype('int') )/2


#[sum(x)==1, ret >= req_return, x >= 0]


premio_inicial = df_lote_otim["Premio_oferecido_com_batente"]*0.9
# limitando o premio fique dentro do range
bounds = [(0.9 * premio_base, premio_base*1.1) for premio_base in df_lote_otim["Premio_oferecido_com_batente"]]



demanda_alvo=0.45

# rodando a otimizaçao
result = minimize(margem_esperada,
                  premio_inicial,
                  args=(demanda_alvo), 
                  method = 'L-BFGS-B', 
                  tol=0.01,
                  constraints=restricoes,
                  bounds=bounds)

# premio otimizados
premio_otim = result.x
df_lote_otim["premio"]=result.x

df_lote_otim["demanda otimizada"] = model.predict(df_lote_otim)
df_lote_otim["Margem otimizada"] = df_lote_otim["demanda otimizada"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])

CPU times: total: 3.31 s
Wall time: 3.57 s


In [454]:
print(np.mean(df_lote_otim["Premio_oferecido_com_batente"]))
print(np.mean(df_lote_otim["premio"]))

3186.1766349890004
2867.871898483556


In [451]:
#df_lote_otim.columns.to_list()

In [452]:
#df_lote_otim2 = df_lote_otim[["chave","Act_cost","Premio_oferecido_com_batente","premio","prob_est","demanda otimizada","Margem Inicial","Margem otimizada","premio_anterior"]]
#df_lote_otim2

In [453]:
print(np.mean(df_lote_otim["Premio_oferecido_com_batente"]))
print(np.mean(df_lote_otim["premio"]))
print(np.mean(df_lote_otim["Margem Inicial"]))
print(np.mean(df_lote_otim["Margem otimizada"]))

print(np.mean(df_lote_otim["prob_est"]))
print(np.mean(df_lote_otim["demanda otimizada"]))


3186.1766349890004
2867.871898483556
447.0520887105169
390.18415765035417
0.4330839286994385
0.4909699526181443


In [413]:
prices_df = pd.DataFrame({
    'chave':         df_lote_otim["chave"],
    'Premio ofertado com batente':  df_lote_otim["Premio_oferecido_com_batente"],
    'Optimized Prices': df_lote_otim["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_lote_otim["prob_est"],
    'renewal demand':df_lote_otim["demanda otimizada"]
})
prices_df.head(20)

,chave,Premio ofertado com batente,Optimized Prices,lb_bound,up_bound,prob_est,renewal demand
7447,600900307043100001,1497.722170,1363.379744,1347.949953,1647.494387,0.584916,0.639774
24233,4001424693100001,4903.842610,4462.491682,4413.458349,5394.226871,0.233444,0.288944
2637,1401214433100001,1346.507593,1225.658062,1211.856834,1481.158352,0.565358,0.624755
13030,613700218223100001,2323.158348,2114.360045,2090.842513,2555.474183,0.507035,0.573077
18722,49009872283100001,3459.435036,3148.296888,3113.491532,3805.378540,0.344950,0.401681
20250,11100379343100001,2793.424549,2542.328481,2514.082094,3072.767004,0.474962,0.532433
5374,10000971993100001,1686.996043,1535.231883,1518.296439,1855.695647,0.155988,0.191795
21339,600700666503100001,3507.645000,3192.258905,3156.880500,3858.409500,0.486512,0.555799
26728,90808135793100001,7189.800156,6542.705773,6470.820140,7908.780172,0.145419,0.176249
2154,85600111863100001,1324.211325,1205.329896,1191.790193,1456.632458,0.433068,0.492210


In [ ]:
# Calculate revenue after optimization
revenue_after = np.mean(optimized_prices)
# Define revenue before and after optimization
revenue_values = [revenue_before, revenue_after]

# Print revenue before and after optimization
print("Revenue before optimization:", revenue_before)
print("Revenue after optimization:", revenue_after)


# Define labels for the bars
labels = ['Revenue Before Optimization', 'Revenue After Optimization']

# Plotting
plt.bar(labels, revenue_values, color=['blue', 'green'])
plt.ylabel('Price')
plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

#criando a função

In [227]:
predicted_prices = df_lote_otim["Premio_oferecido_com_batente"]
revenue_before = df_lote_otim["Premio_oferecido_com_batente"]

In [228]:
def objective_function(prices):
    revenue = -np.sum(prices)  
    return revenue

# Define constraints
def constraint(prices, predicted_prices, min_revenue):
    return np.concatenate([prices - predicted_prices, predicted_prices * 1.2 - prices, [np.sum(prices) - min_revenue]])

# Define initial prices (use predicted prices as initial guess)
initial_prices = df_lote_otim["Premio_oferecido_com_batente"]*0.9

# Minimum revenue constraint
min_revenue = np.sum(predicted_prices) * 0.9  


# Adjust constraints to ensure prices are no lower than predicted and no greater than 1.2 times predicted
bounds = [(price, price * 1.2) for price in predicted_prices]

# Perform optimization
result = minimize(objective_function, 
                  initial_prices,
                  constraints={'type': 'ineq', 'fun': constraint,
                  'args': (predicted_prices, min_revenue)}, 
                   bounds=bounds)

# Get optimized prices
optimized_prices = result.x

# Calculate revenue after optimization
revenue_after = np.sum(optimized_prices)

# Print revenue before and after optimization
print("Revenue before optimization:", revenue_before)
print("Revenue after optimization:", revenue_after)

Revenue before optimization: 22272    4689.450000
9739     2128.848354
14473    2012.412374
8662     1946.558922
7284     1662.238362
13247    2354.677691
15216    1661.696036
4415     1548.016372
15167    2487.958735
11423    2298.231035
Name: Premio_oferecido_com_batente, dtype: float64
Revenue after optimization: 27348.105456977137


In [232]:
print(optimized_prices),

[5627.33999997 2554.61802478 2414.89484878 2335.87070638 1994.68603438
 2825.61322918 1994.03524318 1857.61964638 2985.55048198 2757.87724198]
